# Лабораторная работа №10. ОСНОВЫ ОБРАБОТКИ ЕСТЕСТВЕННОГО ЯЗЫКА (NLP). ЗАДАЧА ТЕМАТИЧЕСКОГО МОДЕЛИРОВАНИЯ

ЗАДАНИЕ
1. Для выполнения задания используйте датасет с данными о спаме (https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset).
2***. Самостоятельно реализовать BoW, TF-IDF.
3. Решить задачу классификации с понижением размерности.
*** Использовать самостоятельно реализованные модели из предыдущих ЛР.
4. Решить задачу тематического моделирования с помощью LDA.

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import numpy as np
from collections import defaultdict
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
data = pd.read_csv('spam.csv', encoding='latin-1')
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [3]:
def clean(text):
    # Удаление специальных символов и приведение к нижнему регистру
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Применение функции очистки к датасету
data['v2'] = data['v2'].apply(clean)

In [4]:
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,go until jurong point crazy available only in ...,NaN,NaN,NaN
1,ham,ok lar joking wif u oni,NaN,NaN,NaN
2,spam,free entry in 2 a wkly comp to win fa cup fina...,NaN,NaN,NaN
3,ham,u dun say so early hor u c already then say,NaN,NaN,NaN
4,ham,nah i don t think he goes to usf he lives arou...,NaN,NaN,NaN
5,spam,freemsg hey there darling it s been 3 week s n...,NaN,NaN,NaN
6,ham,even my brother is not like to speak with me t...,NaN,NaN,NaN
7,ham,as per your request melle melle oru minnaminun...,NaN,NaN,NaN
8,spam,winner as a valued network customer you have b...,NaN,NaN,NaN
9,spam,had your mobile 11 months or more u r entitled...,NaN,NaN,NaN


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data['v2'], data['v1'], test_size=0.2, random_state=42)

# 1. Самостоятельно реализовать BoW, TF-IDF.

In [6]:
def create_vocabulary(texts):
    vocab = set()
    for text in texts:
        vocab.update(text.split())
    return sorted(vocab)

# BoW
def bow(texts, vocabulary):
    bow_vectors = []
    vocab_indices = {word: i for i, word in enumerate(vocabulary)}
    for text in texts:
        tokenized = text.split()
        bow_vector = np.zeros(len(vocabulary))
        for token in tokenized:
            if token in vocab_indices:
                bow_vector[vocab_indices[token]] += 1
        bow_vectors.append(bow_vector)
    return np.array(bow_vectors)

# TF-IDF
def tfidf(texts, vocabulary):
    document_counts = defaultdict(int)
    for text in texts:
        unique_tokens = set(text.split())
        for token in unique_tokens:
            document_counts[token] += 1

    tfidf_vectors = []
    for text in texts:
        tokenized = text.split()
        tfidf_vector = np.zeros(len(vocabulary))
        for i, word in enumerate(vocabulary):
            if word in tokenized:
                tf = tokenized.count(word) / len(tokenized)
                idf = np.log(len(texts) / (1 + document_counts[word]))
                tfidf_vector[i] = tf * idf
        tfidf_vectors.append(tfidf_vector)
    return np.array(tfidf_vectors)

In [7]:
# Создание словаря и применение методов BoW и TF-IDF
vocabulary = create_vocabulary(X_train)
bow_train = bow(X_train, vocabulary)
tfidf_train = tfidf(X_train, vocabulary)

# Просмотр размеров полученных векторов
bow_train.shape, tfidf_train.shape

((4457, 7775), (4457, 7775))

Сравнение с оригинальными методами 

In [9]:
data['v2'] = data['v2'].apply(clean)

# Разбиение данных на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(data['v2'], data['v1'], test_size=0.2, random_state=42)

# Применение CountVectorizer для BoW
count_vectorizer = CountVectorizer()
bow_train = count_vectorizer.fit_transform(X_train)
bow_test = count_vectorizer.transform(X_test)

# Применение TfidfVectorizer для TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# Получение размеров векторов
print("BoW Train Shape:", bow_train.shape)
print("TF-IDF Train Shape:", tfidf_train.shape)

BoW Train Shape: (4457, 7735)
TF-IDF Train Shape: (4457, 7735)


# 2. Решить задачу с понижением размерности.

In [11]:
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(tfidf_train.toarray())
X_test_pca = pca.transform(tfidf_test.toarray())

# Обучение классификатора (я взял логистическую регрессию)
classifier = LogisticRegression()
classifier.fit(X_train_pca, y_train)

# Предсказание на тестовых данных и вывод результатов
predictions = classifier.predict(X_test_pca)
report = classification_report(y_test, predictions)

def format_classification_report(report):
    formatted_report = ""
    for line in report.split('\n'):
        formatted_report += line + "\n"
    return formatted_report

# Форматированный отчет о классификации
formatted_report = format_classification_report(report)
print(formatted_report)

              precision    recall  f1-score   support

         ham       0.94      1.00      0.97       965
        spam       0.98      0.62      0.76       150

    accuracy                           0.95      1115
   macro avg       0.96      0.81      0.86      1115
weighted avg       0.95      0.95      0.94      1115




# 3. Решить задачу тематического моделирования с помощью LDA.

In [15]:
# Выбор числа тем
n_topics = 5

# Векторизация текста
vectorizer = CountVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(data['v2'])

# Создание и обучение модели LDA
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X_vectorized)

# Получение тем и слов
words = vectorizer.get_feature_names_out()
topics = lda.components_

# Вывод тем с их словами
for topic_idx, topic in enumerate(topics):
    top_words_indices = topic.argsort()[-10:][::-1]  # Индексы слов с наибольшим весом в теме
    top_words = [words[i] for i in top_words_indices]
    print(f"Тема {topic_idx + 1}: {' '.join(top_words)}")

Тема 1: you to and my me the your day love good
Тема 2: you to are call me how have what can do
Тема 3: to free ur now call lor txt ok no or
Тема 4: to you it the in and that be on will
Тема 5: the is to your you in for lt gt and
